# Infersent based Simalirity

In [1]:
from models import InferSent
import torch
import nltk
import numpy as np
from scipy import spatial

### Note this does require downloading GloVe to some folder
Download Glove via
```sh
mkdir GloVe
curl -Lo GloVe/glove.840B.300d.zip http://nlp.stanford.edu/data/glove.840B.300d.zip
unzip GloVe/glove.840B.300d.zip -d GloVe/
```
and the encoders
```sh
mkdir encoder
curl -Lo encoder/infersent1.pkl https://dl.fbaipublicfiles.com/infersent/infersent1.pkl
```
^Modify the Above for your operating system

In [12]:
with open("samples.txt") as f:
    sentences = f.readlines()
with open("triggers.txt") as f:
    triggers = f.readlines()

In [22]:
# Replace with your own path
W2V_PATH = f"GloVe/glove.840B.300d.txt"
model_version = 1
MODEL_PATH = f'encoder/infersent%s.pkl' % model_version

def generate_Infersent_model():
    # Load model
    params_model = {'bsize': 64, 
                    'word_emb_dim': 300, 
                    'enc_lstm_dim': 2048,
                    'pool_type': 'max',
                    'dpout_model': 0.0, 
                    'version': model_version}
    model = InferSent(params_model)
    model.load_state_dict(torch.load(MODEL_PATH))

    model.set_w2v_path(W2V_PATH)

    model.build_vocab_k_words(K=100000)
    model.update_vocab(sentences)
    return model

def get_doc2vec(text, model, verbose=False):
    emb = model.encode(text, verbose=verbose)
    return emb


In [23]:
model = generate_Infersent_model()

# Run the following if saved
# PATH = (os.path.dirname(os.path.abspath(__file__))) + "/"
# model = torch.load(PATH + "infraset_model.torch")
# model.eval()

Vocab size : 100000
Found 819(/1017) words with w2v vectors
New vocab size : 100822 (added 819 words)


In [27]:
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

res = get_doc2vec(["happy", "camper camper"], model)




array([-0.01988107, -0.08939895, -0.0304451 , ..., -0.03926746,
       -0.03814262, -0.0289226 ], dtype=float32)

In [80]:
from tqdm import tqdm
import random

def compute_accuracy(sentences, triggers, is_trigger):
    identified_trigger = 0
    random.shuffle(sentences)
    for sent in tqdm(sentences):
        for trig in triggers:
            if is_trigger(sent, trig):
                identified_trigger += 1
                break
    return identified_trigger, identified_trigger/len(sentences)


In [79]:
def is_trigger(sent, trig):
    res = get_doc2vec([sent, trig], model)
    if cosine(res[0], res[1]) > .8:
        return True
    return False
compute_accuracy(sentences, triggers, is_trigger)

  0%|          | 2/9815 [00:23<31:42:40, 11.63s/it]


KeyboardInterrupt: 

In [78]:
compute_accuracy(triggers, triggers, is_trigger)

100%|██████████| 37/37 [01:36<00:00,  2.62s/it]


(37, 1.2333333333333334)

In [ ]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/1?tf-hub-format=compressed"

# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url)
similarity_input_placeholder = tf.placeholder(tf.string, shape=(None))
similarity_message_encodings = embed(similarity_input_placeholder)


In [41]:
res = get_doc2vec(["The beaches are very dirty in Mumbai", "What will be the weather be?"], model)

In [42]:
cosine(res[0], res[1])

0.394147

In [43]:
res = get_doc2vec(["is the weather going to get hotter?", "What will be the weather be?"], model)

In [44]:
cosine(res[0], res[1])

0.77164006

In [45]:
len(sentences)

9815

In [84]:
import spacy
nlp = spacy.load('en_core_web_lg')


OSError: [E050] Can't find model 'en_core_web_lg'. It doesn't seem to be a Python package or a valid path to a data directory.